In [1]:
# !pip install xarray 
# !pip install netcdf4

import os
import xarray as xr
import pandas as pd

input_folder = r"C:\DataThesis\ERA5_Monthly"
output_folder = r"C:\DataThesis\ERA5_Monthly"


os.makedirs(output_folder, exist_ok=True)

for file in os.listdir(input_folder):
    if file.endswith('.nc'):
        file_path = os.path.join(input_folder, file)
        print(f"Processing: {file}")
        try:
            # Gunakan engine netcdf4 untuk NetCDF4 files
            ds = xr.open_dataset(file_path, engine='netcdf4')
            df = ds.to_dataframe().reset_index()
            csv_filename = os.path.splitext(file)[0] + '.csv'
            df.to_csv(os.path.join(output_folder, csv_filename), index=False)
            print(f"Saved: {csv_filename}")
        except Exception as e:
            print(f"Failed to process {file}: {e}")

Processing: data_era5_hourly_2022_02.nc
Failed to process data_era5_hourly_2022_02.nc: [Errno -51] NetCDF: Unknown file format: 'C:\\DataThesis\\ERA5_Monthly\\data_era5_hourly_2022_02.nc'


In [ ]:
import cdsapi

# --- 1. ISI BAGIAN INI ---
# Ganti string di bawah dengan API key lengkap Anda.
# Dapatkan dari: https://cds.climate.copernicus.eu/user/profile
# Key Anda akan terlihat seperti: "12345:abcdef-1234-abcd-1234-abcdef123456"
CDS_API_KEY = "0454455d-cef6-4a4e-ad46-f0e905629467"
# -------------------------

# Definisikan URL API
CDS_API_URL = "https://cds.climate.copernicus.eu/api"

# Inisialisasi CDS API client secara manual
# Script ini akan mengabaikan file .cdsapirc dan menggunakan key di atas
c = cdsapi.Client(url=CDS_API_URL, key=CDS_API_KEY)

# --- 2. SISA SKRIP ANDA (Tidak perlu diubah) ---
print("Memulai request ke CDS...")

# Melakukan request data
c.retrieve(
    # Nama dataset yang kita gunakan
    'reanalysis-era5-single-levels',
    {
        # Tentukan format (NetCDF adalah yang paling umum untuk Python)
        'product_type': 'reanalysis',
        'format': 'netcdf',
        
        # Ini adalah daftar variabel yang kita butuhkan
        'variable': [
            '10m_u_component_of_wind', '10m_v_component_of_wind', # Untuk Wind Speed
            '2m_dewpoint_temperature', '2m_temperature',          # Untuk Suhu, Titik Embun, dan Kelembapan
            'total_precipitation',                               # Untuk Curah Hujan
        ],
        
        # Tentukan Waktu (Contoh: 1 hari penuh di tahun 2023)
        'year': '2023',
        'month': '01',
        'day': '01',
        'time': [
            '00:00', '01:00', '02:00', '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00', '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00', '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00', '21:00', '22:00', '23:00',
        ],
        
        # Tentukan Area [North, West, South, East]
        # Ini adalah contoh bounding box kecil di sekitar Jakarta
        'area': [
            -6.1, 106.7, -6.3, 106.9,
        ],
    },
    # Nama file output
    'data_era5_jakarta.nc')

print("Download selesai. File disimpan sebagai 'data_era5_jakarta.nc'")

In [ ]:
import cdsapi
import xarray as xr
import numpy as np
import metpy.calc as mpcalc
from metpy.units import units
import pandas as pd
import os
import zipfile
import shutil
import warnings

# Menonaktifkan peringatan runtime yang tidak relevan
warnings.filterwarnings("ignore", category=RuntimeWarning) 

# --- 1. ISI BAGIAN INI ---
# !! PENTING: Ganti dengan API key BARU Anda. Key Anda sebelumnya sudah terekspos.
# Kunjungi: https://cds.climate.copernicus.eu/user/profile
CDS_API_KEY = "0454455d-cef6-4a4e-ad46-f0e905629467"
# -------------------------

# --- 2. PENGATURAN FILE & LOKASI ---
CDS_API_URL = "https://cds.climate.copernicus.eu/api"
DOWNLOAD_FILENAME = 'data_era5_hourly_2022_2024.zip' # Kita download sebagai ZIP
EXTRACT_FOLDER = 'temp_era5_data'                   # Folder sementara untuk file .nc
CSV_FILENAME = 'data_era5_DAILY_2022_2024_final.csv'

# Area [North, West, South, East] - Area Jakarta
AREA = [-6.1, 106.7, -6.3, 106.9]

# Inisialisasi CDS API client
c = cdsapi.Client(url=CDS_API_URL, key=CDS_API_KEY)

# --- 3. FUNGSI UTAMA ---

def download_data():
    """
    Mengunduh data 3 tahun sebagai file ZIP.
    Melewatkan jika file sudah ada.
    """
    if os.path.exists(DOWNLOAD_FILENAME):
        print(f"File '{DOWNLOAD_FILENAME}' sudah ada. Melewatkan proses download.")
        return True

    print(f"Memulai request data 3 tahun (2022-2024) ke CDS.")
    print("PERINGATAN: Ini adalah request data yang SANGAT BESAR.")
    print("Proses ini bisa memakan waktu BEBERAPA JAM di antrian server CDS.")
    
    try:
        c.retrieve(
            'reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'variable': [
                    '10m_u_component_of_wind', '10m_v_component_of_wind',
                    '2m_dewpoint_temperature', '2m_temperature',
                    'total_precipitation',
                ],
                'year': ['2022', '2023', '2024'],
                'month': [
                    '01', '02', '03', '04', '05', '06',
                    '07', '08', '09', '10', '11', '12'
                ],
                'day': [
                    '01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
                    '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
                    '21', '22', '23', '24', '25', '26', '27', '28', '29', '30',
                    '31'
                ],
                'time': 'all', # 'all' adalah pintasan untuk 24 jam
                'area': AREA,
                'format': 'zip', # --- PENTING: Minta sebagai ZIP ---
            },
            DOWNLOAD_FILENAME
        )
        print(f"Download data HOURLY selesai. File disimpan sebagai '{DOWNLOAD_FILENAME}'")
        return True
    except Exception as e:
        print(f"Terjadi error saat download: {e}")
        print("Pastikan API Key Anda benar dan Anda sudah menyetujui Terms & Conditions di situs CDS.")
        return False

def extract_data():
    """
    Mengekstrak file ZIP ke folder sementara.
    """
    print(f"Mengekstrak file '{DOWNLOAD_FILENAME}' ke '{EXTRACT_FOLDER}'...")
    try:
        # Buat folder jika belum ada
        if os.path.exists(EXTRACT_FOLDER):
            shutil.rmtree(EXTRACT_FOLDER) # Hapus folder lama jika ada
        os.makedirs(EXTRACT_FOLDER)
        
        # Ekstrak file ZIP
        with zipfile.ZipFile(DOWNLOAD_FILENAME, 'r') as zip_ref:
            zip_ref.extractall(EXTRACT_FOLDER)
        print("Ekstrak selesai.")
        return True
    except Exception as e:
        print(f"Gagal mengekstrak file ZIP: {e}")
        return False

def process_to_csv():
    """
    Memproses semua file .nc di folder, mengagregasi harian,
    dan menyimpan sebagai satu CSV.
    """
    print(f"Memproses file .nc dari folder '{EXTRACT_FOLDER}'...")
    try:
        # Membuka SEMUA file .nc di folder sebagai satu dataset
        # Ini akan otomatis menggabungkan file 'instant' dan 'accumulated'
        ds = xr.open_mfdataset(
            os.path.join(EXTRACT_FOLDER, '*.nc'), 
            engine='netcdf4', 
            combine='by_coords'
        )

        # --- A. KALKULASI PER JAM ---
        print("...Menghitung variabel per jam (Wind Speed, Humidity, Celsius)...")
        
        # 1. Konversi Suhu (t2m) & Titik Embun (d2m) ke Celsius
        ds['t2m_celsius'] = ds['t2m'] - 273.15
        ds['t2m_celsius'].attrs['units'] = 'C'
        ds['d2m_celsius'] = ds['d2m'] - 273.15
        ds['d2m_celsius'].attrs['units'] = 'C'

        # 2. Kalkulasi Kelembapan (Relative Humidity)
        t_dew = ds['d2m'].metpy.quantify()
        t_air = ds['t2m'].metpy.quantify()
        rh_hourly = mpcalc.relative_humidity_from_dewpoint(t_air, t_dew)
        ds['relative_humidity'] = rh_hourly.metpy.dequantify()
        ds['relative_humidity'].attrs['units'] = 'fraction'

        # 3. Kalkulasi Kecepatan Angin (Wind Speed)
        ds['wind_speed_10m'] = np.sqrt(ds['u10']**2 + ds['v10']**2)
        ds['wind_speed_10m'].attrs['units'] = 'm s**-1'

        # --- B. AGREGRASI MENJADI HARIAN ---
        print("...Mengubah data per jam menjadi data HARIAN...")

        # Agregasi Rata-rata Harian (mean)
        daily_temp_mean = ds['t2m_celsius'].resample(time='D').mean()
        daily_dewpoint_mean = ds['d2m_celsius'].resample(time='D').mean()
        daily_humidity_mean = ds['relative_humidity'].resample(time='D').mean()
        daily_wind_speed_mean = ds['wind_speed_10m'].resample(time='D').mean()
        
        # Agregasi Total Harian (sum)
        daily_precip_sum = ds['tp'].resample(time='D').sum()
        daily_precip_sum.attrs['units'] = 'm of water per day' # total meter per hari

        # --- C. MENGGABUNGKAN DAN MENYIMPAN CSV ---
        print("Menggabungkan semua data harian...")
        
        daily_ds = xr.Dataset({
            'temperature_celsius_mean': daily_temp_mean,
            'dewpoint_celsius_mean': daily_dewpoint_mean,
            'relative_humidity_percent_mean': daily_humidity_mean * 100, # Konversi ke %
            'wind_speed_ms_mean': daily_wind_speed_mean,
            'precipitation_m_per_day_sum': daily_precip_sum
        })

        print("Mengonversi data ke DataFrame...")
        df = daily_ds.to_dataframe()
        df = df.reset_index() # Membuat lat, lon, time menjadi kolom

        print(f"Menyimpan data ke {CSV_FILENAME}...")
        df.to_csv(CSV_FILENAME, index=False, date_format='%Y-%m-%d')
        
        print("\n--- BERHASIL! ---")
        print(f"Data HARIAN (daily) untuk 2022-2024 telah disimpan sebagai '{CSV_FILENAME}'")
        return True

    except Exception as e:
        print(f"Terjadi error saat memproses file .nc: {e}")
        return False
    finally:
        # Selalu tutup dataset
        try:
            ds.close()
        except:
            pass

def cleanup():
    """
    Membersihkan file sementara.
    """
    print("Membersihkan file sementara...")
    try:
        if os.path.exists(EXTRACT_FOLDER):
            shutil.rmtree(EXTRACT_FOLDER)
            print(f"Folder '{EXTRACT_FOLDER}' telah dihapus.")
        
        # (Opsional) Hapus file ZIP asli untuk hemat ruang
        # if os.path.exists(DOWNLOAD_FILENAME):
        #     os.remove(DOWNLOAD_FILENAME)
        #     print(f"File '{DOWNLOAD_FILENAME}' telah dihapus.")
            
    except Exception as e:
        print(f"Error saat membersihkan file: {e}")

# --- 4. MENJALANKAN SKRIP ---
if __name__ == "__main__":
    if download_data():
        if extract_data():
            if process_to_csv():
                cleanup()

In [ ]:
import cdsapi
import xarray as xr
import numpy as np
import metpy.calc as mpcalc
from metpy.units import units
import pandas as pd
import os
import shutil
import warnings

# Menonaktifkan peringatan runtime yang tidak relevan
warnings.filterwarnings("ignore", category=RuntimeWarning) 

# --- 1. ISI BAGIAN INI ---
# !! GANTI DENGAN API KEY BARU ANDA !!
CDS_API_KEY = "0454455d-cef6-4a4e-ad46-f0e905629467"
# -------------------------

# --- 2. PENGATURAN FILE & LOKASI ---
CDS_API_URL = "https://cds.climate.copernicus.eu/api" # URL API v2 yang benar
DOWNLOAD_FILENAME = 'data_era5_hourly_2022_2024.nc' # Langsung simpan sebagai .nc
CSV_FILENAME = 'data_era5_DAILY_2022_2024_final.csv'

# Area [North, West, South, East] - Area Jakarta
AREA = [-6.1, 106.7, -6.3, 106.9]

# Inisialisasi CDS API client
c = cdsapi.Client(url=CDS_API_URL, key=CDS_API_KEY)

# --- 3. FUNGSI UTAMA ---

def download_data():
    """
    Mengunduh data 3 tahun sebagai satu file NetCDF.
    Melewatkan jika file sudah ada.
    """
    if os.path.exists(DOWNLOAD_FILENAME):
        print(f"File '{DOWNLOAD_FILENAME}' sudah ada. Melewatkan proses download.")
        return True

    print(f"Memulai request data 3 tahun (2022-2024) ke CDS.")
    print("PERINGATAN: Ini adalah request data yang SANGAT BESAR.")
    print("Proses ini bisa memakan waktu BEBERAPA JAM di antrian server CDS.")
    
    try:
        c.retrieve(
            'reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'variable': [
                    '10m_u_component_of_wind', '10m_v_component_of_wind',
                    '2m_dewpoint_temperature', '2m_temperature',
                    'total_precipitation',
                ],
                'year': ['2022'
                        # , '2023', '2024'
                         ],
                'month': [
                    '01'
                    #, '02', '03', '04', '05', '06',
                    #'07', '08', '09', '10', '11', '12'
                ],
                'day': [
                    '01', '02', '03', '04', '05', '06', '07', '08', '09', '10'
                    #,
                    #'11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
                    #'21', '22', '23', '24', '25', '26', '27', '28', '29', '30',
                    #'31'
                ],
                'time': 'all', 
                'area': AREA,
                'format': 'netcdf', # --- PERBAIKAN: Minta sebagai NetCDF ---
            },
            DOWNLOAD_FILENAME
        )
        print(f"Download data HOURLY selesai. File disimpan sebagai '{DOWNLOAD_FILENAME}'")
        return True
    except Exception as e:
        print(f"Terjadi error saat download: {e}")
        print("Pastikan API Key Anda benar dan Anda sudah menyetujui Terms & Conditions di situs CDS.")
        return False

# --- FUNGSI extract_data() DIHAPUS KARENA TIDAK DIPERLUKAN LAGI ---

def process_to_csv():
    """
    Memproses file .nc yang diunduh, mengagregasi harian,
    dan menyimpan sebagai satu CSV.
    """
    print(f"Memproses file .nc '{DOWNLOAD_FILENAME}'...")
    try:
        # --- PERBAIKAN: Langsung buka file .nc yang diunduh ---
        ds = xr.open_dataset(DOWNLOAD_FILENAME, engine='netcdf4')

        # --- A. KALKULASI PER JAM ---
        print("...Menghitung variabel per jam (Wind Speed, Humidity, Celsius)...")
        
        # 1. Konversi Suhu (t2m) & Titik Embun (d2m) ke Celsius
        ds['t2m_celsius'] = ds['t2m'] - 273.15
        ds['t2m_celsius'].attrs['units'] = 'C'
        ds['d2m_celsius'] = ds['d2m'] - 273.15
        ds['d2m_celsius'].attrs['units'] = 'C'

        # 2. Kalkulasi Kelembapan (Relative Humidity)
        t_dew = ds['d2m'].metpy.quantify()
        t_air = ds['t2m'].metpy.quantify()
        rh_hourly = mpcalc.relative_humidity_from_dewpoint(t_air, t_dew)
        ds['relative_humidity'] = rh_hourly.metpy.dequantify()
        ds['relative_humidity'].attrs['units'] = 'fraction'

        # 3. Kalkulasi Kecepatan Angin (Wind Speed)
        ds['wind_speed_10m'] = np.sqrt(ds['u10']**2 + ds['v10']**2)
        ds['wind_speed_10m'].attrs['units'] = 'm s**-1'

        # --- B. AGREGRASI MENJADI HARIAN ---
        print("...Mengubah data per jam menjadi data HARIAN...")

        # Agregasi Rata-rata Harian (mean)
        daily_temp_mean = ds['t2m_celsius'].resample(time='D').mean()
        daily_dewpoint_mean = ds['d2m_celsius'].resample(time='D').mean()
        daily_humidity_mean = ds['relative_humidity'].resample(time='D').mean()
        daily_wind_speed_mean = ds['wind_speed_10m'].resample(time='D').mean()
        
        # Agregasi Total Harian (sum)
        daily_precip_sum = ds['tp'].resample(time='D').sum()
        daily_precip_sum.attrs['units'] = 'm of water per day' 

        # --- C. MENGGABUNGKAN DAN MENYIMPAN CSV ---
        print("Menggabungkan semua data harian...")
        
        daily_ds = xr.Dataset({
            'temperature_celsius_mean': daily_temp_mean,
            'dewpoint_celsius_mean': daily_dewpoint_mean,
            'relative_humidity_percent_mean': daily_humidity_mean * 100, # Konversi ke %
            'wind_speed_ms_mean': daily_wind_speed_mean,
            'precipitation_m_per_day_sum': daily_precip_sum
        })

        print("Mengonversi data ke DataFrame...")
        df = daily_ds.to_dataframe()
        df = df.reset_index() # Membuat lat, lon, time menjadi kolom

        print(f"Menyimpan data ke {CSV_FILENAME}...")
        df.to_csv(CSV_FILENAME, index=False, date_format='%Y-%m-%d')
        
        print("\n--- BERHASIL! ---")
        print(f"Data HARIAN (daily) untuk 2022-2024 telah disimpan sebagai '{CSV_FILENAME}'")
        return True

    except Exception as e:
        print(f"Terjadi error saat memproses file .nc: {e}")
        # Jika errornya [Errno -51], itu adalah error "Terms & Conditions"
        if "[Errno -51]" in str(e):
             print("\n--- ERROR PENTING ---")
             print("Error [Errno -51] berarti file yang diunduh adalah file error HTML.")
             print("SOLUSI: Buka situs CDS, login, dan 'Accept' Terms & Conditions untuk dataset 'reanalysis-era5-single-levels'.")
        return False
    finally:
        try:
            ds.close()
        except:
            pass

def cleanup():
    """
    Membersihkan file .nc besar setelah CSV dibuat.
    """
    print("Membersihkan file sementara...")
    try:
        if os.path.exists(DOWNLOAD_FILENAME):
            os.remove(DOWNLOAD_FILENAME)
            print(f"File '{DOWNLOAD_FILENAME}' telah dihapus untuk menghemat ruang.")
    except Exception as e:
        print(f"Error saat membersihkan file: {e}")

# --- 4. MENJALANKAN SKRIP ---
if __name__ == "__main__":
    if download_data():
        if process_to_csv():
            cleanup() # Hapus file .nc yang besar setelah CSV berhasil dibuat